## HIV News articles extraction from The Daily Star. Data Extraction of following parameters
- Headline
- Description
- Author
- Published_Date
- News
- URL
- Keywords
- Summary

### Importing the necessary Libraries

In [1]:
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

### Creating Empty lists for HIV News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls = [], [], [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from google search results¶

In [3]:
keyword = 'HIV site:www.thedailystar.net'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

soup = BeautifulSoup(get(url).text, 'lxml')
try:
    # Extracts the digits if it the resulted number without comma ','. eg: About 680 results (0.23 seconds)
    max_pages = round([int(s) for s in soup.select_one('div#resultStats').text.split() if s.isdigit()][0]/10)
    max_pages = max_pages + 1
except:
    # Extracts the digits if it the resulted number without comma ','. eg: About 1,080 results (0.23 seconds)
    max_pages = round(int(''.join(i for i in soup.select_one('div#resultStats').text if i.isdigit()))/10)
    max_pages = max_pages + 1

### Iterates max_pages value through while loop. Scraping the Articles urls

In [4]:
options = Options()
options.headless = True
browser = webdriver.Chrome(options=options)
browser.get(url)

index = 0

while True:
    try:
        index +=1
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        if index == max_pages:
            break
        browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
        time.sleep(2)
        sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
        sys.stdout.flush()
    except:
        pass
    
browser.quit()

### To remove duplicates urls entries in the list by executing below line

In [5]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

286 <class 'list'>


### Iterates urls through for loop. Scraping the Articles with above parameters

In [6]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()
        
        # Extracts the Headlines
        try:
            try:
                headlines.append(soup.select_one('meta[property="og:title"]')['content'].strip())
            except:
                headlines.append(article.title.strip())
        except:
            headlines.append(None)
            
        # Extracts the Descriptions    
        try:
            try:
                descriptions.append(soup.select_one('meta[property="og:description"]')['content'].strip().replace('\n', ' '))
            except:
                descriptions.append(article.meta_description.strip())
        except:
            descriptions.append(None)
            
        # Extracts the Authors
        try:
            try:
                authors.append(soup.select_one('.detailed-content').select_one('span[itemprop="name"]').a.text.strip())
            except:
                authors.append(article.authors.strip())
        except:
            authors.append(None)
        
        # Extracts the published dates
        try:
            try:
                dates.append(soup.select_one('meta[property="article:published_time"]')['content'].strip())
            except:
                dates.append(str(article.publish_date))
        except:
            dates.append(None)
            
        # Extracts the news articles
        try:
            try:
                news.append(soup.select_one('.node-content').text.replace('\n', '').strip())
            except:
                news.append(article.text.strip())
        except:
            news.append(None)
            
        # Extracts Keywords and Summaries    
        try:
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 31min 44sedailystar.net/news/media-can-dispel-ostracism91prevention-1221439sh-130918-todaygnostic-tests-1581874un-1436092


### Checking Array Length of each list to create DataFrame

In [7]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls))

286 286 286 286 286 286 286 286


### Creating a csv file after checking array length and droping the missing values from the dataset

In [8]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,})
#tbl = tbl.dropna()
tbl.to_csv('The_Daily_Star.csv', index=False)
tbl.head()

Headlines  \
0    869 diagnosed with HIV in the country this year   
1   HIV infects one teenage girl every 3 minutes: UN   
2       Like a phoenix: living with HIV for 32 years   
3  London HIV patient becomes world's second AIDS...   
4                HIV deaths: Bangladesh 10th in Asia   

                                        Descriptions         Authors  \
0  A total of 869 people affected by Human Immuno...            None   
1  Every three minutes, a girl between the ages o...            None   
2  It was the hottest day of summer Europe ever e...  Dr Maruf Hasan   
3  An HIV-positive man in Britain has become the ...            None   
4  Bangladesh stands at the tenth position in HIV...            None   

             Published_Dates  \
0  2018-12-02T00:00:00+06:00   
1  2018-07-26T11:16:02+06:00   
2  2018-11-18T00:00:00+06:00   
3  2019-03-05T16:41:04+06:00   
4  2018-01-26T11:23:58+06:00   

                                            Articles  \
0  A total of 869 people affected by Human Immuno...   
1  Every three minutes, a girl between the ages o...   
2  It was the hottest day of summer Europe ever e...   
3  An HIV-positive man in Britain has become the ...   
4  Photo: DataLEADS/ Asia News NetworkBangladesh ...   

                                            Keywords  \
0  [869, bangladesh, patients, aids, virus, healt...   
1  [girl, 15, girls, sex, epidemic, 19, infected,...   
2  [usa, phoenix, worked, treated, treatment, hae...   
3  [second, donor, case, patient, london, aids, m...   
4  [2016, disease, related, bangladesh, infection...   

                                           Summaries  
0  A total of 869 people affected by Human Immuno...  
1  Girls and young women made up two-thirds of 15...  
2  We were there to talk about decriminalisation ...  
3  We can't detect anything," said Ravindra Gupta...  
4  Bangladesh stands at the tenth position in HIV...